### LSTM

In [1]:
# !pip install mido

In [2]:
# !pip install keras
# import keras

In [2]:
# Library to process MIDI files
import tensorflow as tf

In [4]:
tf.__version__

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-65996cac02ec>", line 1, in <module>
    tf.__version__
AttributeError: module 'tensorflow' has no attribute '__version__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_modul

AttributeError: module 'tensorflow' has no attribute '__version__'

In [5]:
from tensorflow.keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "/Users/sean/opt/anaconda3/lib/python3.7/imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Users/sean/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Users/sean/opt/anacon

TypeError: can only concatenate str (not "list") to str

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message

In [ ]:
import os

folder_name = '1-Calming'
keyword = 'A Presence Felt'
for fname in os.listdir('midi/'+folder_name):
    if keyword in fname:
        print(fname, "has the keyword")

In [ ]:
# define which song we are using
folder_name = '1-Calming'
song_name = 'ES_A Presence Felt - Gavin Luke'
midi_filename = 'midi/' + folder_name + '/' + song_name + '.mp3.mid'

midi = MidiFile(midi_filename)

cnt = 0
for msg in midi:
  print(msg)
  cnt+=1
  if cnt>100:
    break 

In [ ]:
# save notes for the song as a list

notes = []
for msg in midi:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1]) # we are just adding the note!
        
print(notes[:15])
print(len(notes))

In [ ]:
## rescale all the notes to the [0,1] interval

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

print(notes[:15])
print(len(notes))

In [ ]:
# predict the next note for the a song sequence

# LSTM layers requires that data must have a certain shape
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
y_test = y[-300:]
y_test = [note[0] for note in y_test]
X = X[:-300]
y = y[:-300]

print(len(X))

In [ ]:
np.shape(X)

In [ ]:
np.shape(y)

In [ ]:
n_prev

In [ ]:
# Define a model

model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)

# train model
model.fit(np.array(X), np.array(y), 
          32, 20, verbose=1)#, callbacks=[model_save_callback])

In [ ]:
# predict notes from test sequence

def get_pred(model, X_test, y_test):
    pred_test = model.predict(np.array(X_test))
    pred = np.squeeze(pred_test)
    pred_test = [note[0] for note in pred_test]
    pred = np.squeeze(scaler.inverse_transform(pred.reshape(-1,1)))
    pred_note = [int(i) for i in pred]
    
    mse = mean_squared_error(pred_test, y_test)

    return pred_test, pred_note, mse

pred_test, pred_note, mse = get_pred(model, X_test, y_test)
# pred_test, pred_note, mse = get_pred(model, X, y)

In [ ]:
# mean squared error for prediction
print(mse)

In [ ]:
# plot the sequences generated

def plot_pred_seq(pred_test, y_test):

    df = pd.DataFrame({'pred_test': pred_test, 'y_test': y_test})

    plt.figure(figsize=(12,5))
    plt.xlabel('Length of Music Sequence')

    ax1 = df.pred_test.plot(color='blue', grid=True, label='pred_test')
    ax2 = df.y_test.plot(color='red', grid=True, secondary_y=True, label='y_test')

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()


    plt.legend(h1+h2, l1+l2, loc=2)
    plt.show()
    
plot_pred_seq(pred_test, y_test)
# plot_pred_seq(pred_test, [note[0] for note in y])

In [ ]:
# generate midi files
mid = MidiFile()
track = MidiTrack()
t = 0
for note in pred_note:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 127]) 
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
    
mid.tracks.append(track)

mid.save('midi_generated/' + folder_name + '/' + song_name + '_gen.mid')

In [ ]:
midi_filename

In [ ]:
## bidirectional lstm

In [ ]:
# model_bi = Sequential()
# model_bi.add(Bidirectional(LSTM(256, return_sequences=True),input_shape=(n_prev, 1)))
# model_bi.add(Dropout(0.3))
# model_bi.add(Bidirectional(LSTM(128, return_sequences=True), 
#                         input_shape=(n_prev, 1)))
# model_bi.add(Dropout(0.3))
# model_bi.add(Bidirectional(LSTM(64, return_sequences=True), 
#                         input_shape=(n_prev, 1)))
# model_bi.add(Dropout(0.3))
# model_bi.add(Dense(1))
# model_bi.add(Activation('sigmoid'))

# optimizer = Adam(lr=0.001)
# model_bi.compile(loss='mse', optimizer=optimizer)

# # train model
# model_bi.fit(np.array(X), np.array(y), 
#           32, 20, verbose=1)#, callbacks=[model_save_callback])

In [ ]:
# pred_test, pred_note, mse = get_pred(model_bi, X_test, y_test)
# print('MSE: ', mse)
# plot_pred_seq(pred_test, y_test)

In [ ]:
song_dict.keys()

In [ ]:
df_X = pd.read_csv('df_X.csv')
df_X = df_X.fillna(0)

df_X.shape

df_y = pd.read_csv('df_y.csv')
df_y.shape

In [ ]:
### Everything ###

import os
song_dict = {}
music_list = pd.read_csv('music_list.csv')
for folder_name in music_list.columns:
#     print(folder_name, '\n')
    song_dict[folder_name] = []
    for fname in os.listdir('midi/'+folder_name):
#         print('fname: ', fname)
        song_dict[folder_name].append(fname)

# print(song_dict['1-Calming'])

# for i_notes in range(df_X.shape[0]):
    
#     folder_name = df_y.iloc[i_notes,:].tolist()[0]
#     song_name = song_dict
i_notes = 0
for folder_name in song_dict.keys():
    for song_name in song_dict[folder_name]:
        if song_name != '.DS_Store':
            print(song_name)
            
            notes = df_X.iloc[i_notes, :].tolist()
            notes = [note for note in notes if note !=0]

            scaler = MinMaxScaler(feature_range=(0,1))
            scaler.fit(np.array(notes).reshape(-1,1))
            notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
            
            
            
#             print(notes)
        #     print(notes)
            # predict the next note for the a song sequence

            # LSTM layers requires that data must have a certain shape
            notes = [list(note) for note in notes]

            # subsample data for training and prediction
            X = []
            y = []
            # number of notes in a batch
            n_prev = 30
            for i in range(len(notes)-n_prev):
                X.append(notes[i:i+n_prev])
                y.append(notes[i+n_prev])
            # save a seed to do prediction later
            X_test = X[-300:]
            y_test = y[-300:]
            y_test = [note[0] for note in y_test]
            X_train = X[:-300]
            y_train = y[:-300]

            ########
            # Define a model
            ########
            model = Sequential()
            model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
            model.add(Dropout(0.3))
            model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
            model.add(Dropout(0.3))
            model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
            model.add(Dropout(0.3))
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            optimizer = Adam(lr=0.001)
            model.compile(loss='mse', optimizer=optimizer)

            # train model
            model.fit(np.array(X), np.array(y), 
                      32, 20, verbose=1)#, callbacks=[model_save_callback])
            pred_test, pred_note, mse = get_pred(model, X_test, y_test)

            plot_pred_seq(pred_test, y_test)

            # generate midi files
            mid = MidiFile()
            track = MidiTrack()
            t = 0
            for note in pred_note:
                # 147 means note_on
                # 67 is velocity
                note = np.asarray([147, note, 127]) 
                bytes = note.astype(int)
                msg = Message.from_bytes(bytes[0:3])
                t += 1
                msg.time = t
                track.append(msg)

            mid.tracks.append(track)

            mid.save('midi_generated/' + folder_name + '/' + song_name + '_gen.mid')

            i_notes += 1

### MusicVAE